<img width="8%" alt="HubSpot.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png" style="border-radius: 15%">

# HubSpot - Score contact from notes
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Score+contact+from+notes:+Error+short+description">Bug report</a>

**Tags:** #hubspot #contact #activity #notes #emails #communications #meetings #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-08-21 (Created: 2023-08-21)

**Description:** This notebook illustrates the process of scoring a contact in HubSpot based on the count of their notes, which correspond to LinkedIn interactions.

**References:**
- [HubSpot API - Contacts](https://developers.hubspot.com/docs/api/crm/contacts)
- [HubSpot API - Associations v4](https://developers.hubspot.com/docs/api/crm/associations)
- [HubSpot API - Communications](https://developers.hubspot.com/docs/api/crm/communications)

## Input

### Import libraries

In [ ]:
import naas
from naas_drivers import hubspot
import requests
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Setup variables
**Mandatory**

[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `cron`: This variable represents the CRON syntax used to run the scheduler. More information here: https://crontab.guru/#0_12,18_*_*_1-5

**Optional**
- `contact_properties`: This is a list of properties that you want to retrieve from your HubSpot contacts.
- `limit`: This variable likely sets a limit to the number of contacts you want to score.

In [ ]:
# Mandatory
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
cron = "0 0 * * 1-5"

# Optional
contact_properties = [
    "hs_object_id",
    "firstname",
    "lastname",
    'email',
    'linkedinbio',
    'jobtitle',
    "notes_last_updated",
]
limit = 1000

## Model

### Get all contacts from HubSpot

In [ ]:
df_contacts = hubspot.connect(hs_access_token).contacts.get_all(contact_properties)
print("Contacts:", len(df_contacts))
df_contacts.head(1)

### Filter on target

In [ ]:
df_target = df_contacts.copy()
df_target = df_target.sort_values(by="notes_last_updated", ascending=False)[:1000].reset_index(drop=True)
print("Targets:", len(df_target))
df_target.head(1)

### Score contact from notes count

In [ ]:
def get_association_from_contact(
    token,
    contact_id,
    endpoint,
):
    # Init
    results = []
    
    # Requests
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    url = f"https://api.hubapi.com/crm/v4/objects/contacts/{contact_id}/associations/{endpoint}"
    
    # Response
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        results = res.json().get("results")
    return results

for row in df_target.itertuples():
    firstname = row.firstname
    lastname = row.lastname
    contact_id = row.hs_object_id
    print(f'Starting with: {firstname} {lastname} ({contact_id})')
    data = get_association_from_contact(
        hs_access_token,
        contact_id,
        "notes"
    )
    print("-> Notes:", len(data))
    update_contact = {"properties": {"score": len(data)}}
    contact = hubspot.connect(hs_access_token).contacts.patch(contact_id, update_contact)

## Output

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)

# Delete scheduler
# naas.scheduler.delete()